In [3]:
import sys
import os

# Add the directory containing your modules to Python path
sys.path.append('/Users/ellateasell/Research/CodonUsageBias/code')

from Bio import SeqIO
from rscu_methods import run_rscu_analysis
from rscu import PROJ_DIR 
import pandas as pd
import numpy as np
from lib.aminoacids import SLOW, FAST

In [4]:
######## differences metrics #############

def sum_of_abs_differences_fast_slow(rscu_w, rscu_b, codons):
    diffs = abs(rscu_w - rscu_b)
    slow_sum = sum(diffs[i] for i, codon in enumerate(codons) if codon in SLOW)
    fast_sum = sum(diffs[i] for i, codon in enumerate(codons) if codon in FAST)
    return slow_sum, fast_sum

def sum_dist_from_line(rscu_w, rscu_b, codons):
    dists = rscu_b - rscu_w
    slow_sum = sum(dists[i] for i, codon in enumerate(codons) if codon in SLOW)
    fast_sum = sum(dists[i] for i, codon in enumerate(codons) if codon in FAST)
    return slow_sum, fast_sum
    

In [ ]:
###############  Maximize Slow codon RSCU and minimize Fast codon RSCU  ###################

UseCs = True
UseSpAsWindow = False
        
# load SP ORFS (genomic)
all_fasta_path = PROJ_DIR / 'data/filtered/dna_filtered.fasta'
records = list(SeqIO.parse(all_fasta_path, 'fasta'))

# load SP data
tsv_path = PROJ_DIR / 'data/filtered/sp_regions_filtered.tsv'
df = pd.read_table(tsv_path, index_col='sys_name')

# build numpy array of cleavage site values corresponding to the ordering of records
arr_cs_aa = np.zeros(len(records))
for i, record in enumerate(records):
    row = df.loc[record.id]
    arr_cs_aa[i] = row['cs'] if UseCs else 0
arr_cs_n = arr_cs_aa * 3

max_diff_slow = 0
best_w_aa_slow = 0
best_d_aa_slow = 0

min_diff_fast = 0
best_w_aa_fast = 0
best_d_aa_fast = 0

for w_aa in range(10, 50, 5): # searching from window size (in amino acids) from 10, 15, 20, ...,50
    for d_aa in range(0, 100, 10): # searching in distance from CS (in amino acids) from 0, 10, 20, ..., 100
        
        w_n = w_aa * 3
        arr_npet_n = np.full(len(records), d_aa * 3, dtype=int)
        
        window_rscu, comp_rscu = run_rscu_analysis(records, arr_cs_n, arr_npet_n, w_n, UseCs, UseSpAsWindow)
        
        rscu_w = np.array(list(window_rscu.values()))
        rscu_b = np.array(list(comp_rscu.values()))
        
        codons = list(window_rscu.keys())
        
        slow_sum, fast_sum = sum_dist_from_line(rscu_w, rscu_b, codons)
        
        if slow_sum > max_diff_slow:
            max_diff_slow = slow_sum
            best_w_aa_slow = w_aa
            best_d_aa_slow = d_aa
            
        if fast_sum < min_diff_fast:
            min_diff_fast = fast_sum
            best_w_aa_fast = w_aa
            best_d_aa_fast = d_aa
            
            
   
print("")
print("------------------------------")
print("Best parameters found for SLOW CODONS:")         
print(f"Best window size (in amino acids): {best_w_aa_slow}, Best distance from CS (in amino acids): {best_d_aa_slow}, Max sum of differences: {max_diff_slow:.3f}")

print("------------------------------")
print("Best parameters found for FAST CODONS:")
print(f"Best window size (in amino acids): {best_w_aa_fast}, Best distance from CS (in amino acids): {best_d_aa_fast}, Min sum of differences: {min_diff_fast:.3f}")

Sequence YCL048W-A too short for window choice.
Invalid extraction for record YCL048W-A. Skipping.
Sequence YBL096C too short for window choice.
Invalid extraction for record YBL096C. Skipping.
Sequence YCL048W-A too short for window choice.
Invalid extraction for record YCL048W-A. Skipping.
Sequence YGL032C too short for window choice.
Invalid extraction for record YGL032C. Skipping.
Sequence YKL096W-A too short for window choice.
Invalid extraction for record YKL096W-A. Skipping.
Sequence YBL096C too short for window choice.
Invalid extraction for record YBL096C. Skipping.
Sequence YCL048W-A too short for window choice.
Invalid extraction for record YCL048W-A. Skipping.
Sequence YGL032C too short for window choice.
Invalid extraction for record YGL032C. Skipping.
Sequence YKL096W-A too short for window choice.
Invalid extraction for record YKL096W-A. Skipping.
Sequence YNL300W too short for window choice.
Invalid extraction for record YNL300W. Skipping.
Sequence YBL096C too short for

In [6]:
###############  Maximize absoslute difference  ###################

UseCs = True
UseSpAsWindow = False
        
# load SP ORFS (genomic)
all_fasta_path = PROJ_DIR / 'data/filtered/dna_filtered.fasta'
records = list(SeqIO.parse(all_fasta_path, 'fasta'))

# load SP data
tsv_path = PROJ_DIR / 'data/filtered/sp_regions_filtered.tsv'
df = pd.read_table(tsv_path, index_col='sys_name')

# build numpy array of cleavage site values corresponding to the ordering of records
arr_cs_aa = np.zeros(len(records))
for i, record in enumerate(records):
    row = df.loc[record.id]
    arr_cs_aa[i] = row['cs'] if UseCs else 0
arr_cs_n = arr_cs_aa * 3

max_diff_slow = 0
best_w_aa_slow = 0
best_d_aa_slow = 0

max_diff_fast = 0
best_w_aa_fast = 0
best_d_aa_fast = 0

max_diff_overall = 0
best_w_aa_overall = 0
best_d_aa_overall = 0

for w_aa in range(10, 50, 5): # searching from window size (in amino acids) from 10, 15, 20, ...,50
    for d_aa in range(0, 100, 10): # searching in distance from CS (in amino acids) from 0, 10, 20, ..., 100
        
        w_n = w_aa * 3
        arr_npet_n = np.full(len(records), d_aa * 3, dtype=int)
        
        window_rscu, comp_rscu = run_rscu_analysis(records, arr_cs_n, arr_npet_n, w_n, UseCs, UseSpAsWindow)
        
        rscu_w = np.array(list(window_rscu.values()))
        rscu_b = np.array(list(comp_rscu.values()))
        
        codons = list(window_rscu.keys())
        
        slow_sum, fast_sum = sum_of_abs_differences_fast_slow(rscu_w, rscu_b, codons)
        
        if slow_sum > max_diff_slow:
            max_diff_slow = slow_sum
            best_w_aa_slow = w_aa
            best_d_aa_slow = d_aa
            
        if fast_sum > max_diff_fast:
            max_diff_fast = fast_sum
            best_w_aa_fast = w_aa
            best_d_aa_fast = d_aa
            
        if slow_sum + fast_sum > max_diff_overall:
            max_diff_overall = slow_sum + fast_sum
            best_w_aa_overall = w_aa
            best_d_aa_overall = d_aa
            
            
   
print("")
print("------------------------------")
print("Best parameters found for SLOW CODONS:")         
print(f"Best window size (in amino acids): {best_w_aa_slow}, Best distance from CS (in amino acids): {best_d_aa_slow}, Max sum of abs differences: {max_diff_slow:.3f}")

print("------------------------------")
print("Best parameters found for FAST CODONS:")
print(f"Best window size (in amino acids): {best_w_aa_fast}, Best distance from CS (in amino acids): {best_d_aa_fast}, Max sum of abs differences: {max_diff_fast:.3f}")

print("------------------------------")
print("Best parameters found overall:")
print(f"Best window size (in amino acids): {best_w_aa_overall}, Best distance from CS (in amino acids): {best_d_aa_overall}, Max sum of abs differences: {max_diff_overall:.3f}")

Sequence YCL048W-A too short for window choice.
Invalid extraction for record YCL048W-A. Skipping.
Sequence YBL096C too short for window choice.
Invalid extraction for record YBL096C. Skipping.
Sequence YCL048W-A too short for window choice.
Invalid extraction for record YCL048W-A. Skipping.
Sequence YGL032C too short for window choice.
Invalid extraction for record YGL032C. Skipping.
Sequence YKL096W-A too short for window choice.
Invalid extraction for record YKL096W-A. Skipping.
Sequence YBL096C too short for window choice.
Invalid extraction for record YBL096C. Skipping.
Sequence YCL048W-A too short for window choice.
Invalid extraction for record YCL048W-A. Skipping.
Sequence YGL032C too short for window choice.
Invalid extraction for record YGL032C. Skipping.
Sequence YKL096W-A too short for window choice.
Invalid extraction for record YKL096W-A. Skipping.
Sequence YNL300W too short for window choice.
Invalid extraction for record YNL300W. Skipping.
Sequence YBL096C too short for